# Imports (compatible for Python 2 and Python 3)

In [1]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

# Mr. Kryaschev's Lab Tasks

In [3]:
query = '''SELECT * FROM fb55.citibike; '''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


## Sort by start_station_id (ascending) and tripduration (descending):

In [4]:
query = '''
SELECT * FROM fb55.citibike 
ORDER BY start_station_id ASC, tripduration DESC;
'''

table = get_data(query)
table.head()

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9115,-73.993929,2099,2015-02-03 10:28:00+00,2015-02-03 11:02:00+00,72,9054,40.767272,328,Watts St & Greenwich St,40.724055,-74.009660,18653,Subscriber,1966.0,2
1,NaN,W 52 St & 11 Ave,NaN,2051,-73.993929,1944,2015-02-01 12:39:00+00,2015-02-01 13:12:00+00,72,1983,40.767272,328,Watts St & Greenwich St,40.724055,-74.009660,17849,Customer,NaN,0
2,NaN,W 52 St & 11 Ave,NaN,40355,-73.993929,1914,2015-02-07 09:49:00+00,2015-02-07 10:21:00+00,72,40341,40.767272,328,Watts St & Greenwich St,40.724055,-74.009660,21520,Subscriber,1966.0,2
3,NaN,W 52 St & 11 Ave,NaN,12996,-73.993929,1801,2015-02-04 06:32:00+00,2015-02-04 07:02:00+00,72,12938,40.767272,328,Watts St & Greenwich St,40.724055,-74.009660,15161,Subscriber,1959.0,2
4,NaN,W 52 St & 11 Ave,NaN,11359,-73.993929,1678,2015-02-03 18:21:00+00,2015-02-03 18:49:00+00,72,11298,40.767272,79,Franklin St & W Broadway,40.719116,-74.006667,21500,Subscriber,1964.0,1


## Select last 10 records of the table (use table's main id field):

In [5]:
query = '''
SELECT * FROM fb55.citibike
ORDER BY cartodb_id DESC
LIMIT 10;
'''

table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 4 St & 2 Ave,NaN,46199,-73.989780,917,2015-02-07 23:59:00+00,2015-02-08 00:15:00+00,439,46200,40.726281,417,Barclay St & Church St,40.712912,-74.010202,20998,Subscriber,1965,2
1,NaN,Carmine St & 6 Ave,NaN,46198,-74.002150,548,2015-02-07 23:58:00+00,2015-02-08 00:08:00+00,368,46199,40.730386,334,W 20 St & 7 Ave,40.742388,-73.997262,19540,Subscriber,1983,2
2,NaN,5 Ave & E 29 St,NaN,46197,-73.986831,392,2015-02-07 23:57:00+00,2015-02-08 00:03:00+00,474,46198,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,15545,Subscriber,1986,1
3,NaN,5 Ave & E 29 St,NaN,46196,-73.986831,428,2015-02-07 23:57:00+00,2015-02-08 00:04:00+00,474,46197,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,16395,Subscriber,1986,2
4,NaN,W 20 St & 8 Ave,NaN,46195,-74.000040,689,2015-02-07 23:57:00+00,2015-02-08 00:08:00+00,470,46196,40.743453,325,E 19 St & 3 Ave,40.736245,-73.984738,15585,Subscriber,1953,1
5,NaN,1 Ave & E 44 St,NaN,46194,-73.969053,1422,2015-02-07 23:57:00+00,2015-02-08 00:20:00+00,455,46195,40.750020,265,Stanton St & Chrystie St,40.722293,-73.991475,20184,Subscriber,1960,2
6,NaN,E 20 St & 2 Ave,NaN,46193,-73.982050,993,2015-02-07 23:56:00+00,2015-02-08 00:13:00+00,461,46194,40.735877,295,Pike St & E Broadway,40.714067,-73.992939,16722,Subscriber,1974,1
7,NaN,Warren St & Church St,NaN,46192,-74.009106,1165,2015-02-07 23:56:00+00,2015-02-08 00:15:00+00,152,46193,40.714740,325,E 19 St & 3 Ave,40.736245,-73.984738,16978,Subscriber,1959,1
8,NaN,W 47 St & 10 Ave,NaN,46191,-73.993012,1248,2015-02-07 23:56:00+00,2015-02-08 00:17:00+00,495,46192,40.762699,432,E 7 St & Avenue A,40.726218,-73.983799,16300,Subscriber,1984,1
9,NaN,W 17 St & 8 Ave,NaN,46190,-74.001497,306,2015-02-07 23:55:00+00,2015-02-08 00:01:00+00,116,46191,40.741776,494,W 26 St & 8 Ave,40.747348,-73.997236,14736,Subscriber,1983,2


## List all unique birth years (hint: distinct):

In [6]:
query = '''
SELECT DISTINCT birth_year FROM fb55.citibike;
'''

table = get_data(query)
table.head()

,birth_year,Unnamed: 1
0,1994.0,NaN
1,1954.0,NaN
2,1979.0,NaN
3,1956.0,NaN
4,1943.0,NaN


## Find minimal, maximal and average trip duration

In [7]:
query = '''
SELECT MIN(tripduration), MAX(tripduration), AVG(tripduration) 
FROM fb55.citibike;
'''

table = get_data(query)
table

,min,max,avg
0,60,43016,675.865823


## Select trips that started at 1am:

In [8]:
query = '''
SELECT starttime FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) =  1
ORDER BY starttime DESC;
'''
# ordered descending to see that only time between 1am and 2am are extracted
table = get_data(query)
table.head()

,starttime,Unnamed: 1
0,2015-02-07 01:58:00+00,NaN
1,2015-02-07 01:58:00+00,NaN
2,2015-02-07 01:55:00+00,NaN
3,2015-02-07 01:55:00+00,NaN
4,2015-02-07 01:55:00+00,NaN


## Find average birth year of people that ride bikes at 2am

In [9]:
query = '''
SELECT AVG(birth_year) FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) = 2
'''

table = get_data(query)
table

,avg,Unnamed: 1
0,1978.201754,NaN


## Age of oldest person riding at 3am

In [10]:
query = '''
SELECT MAX(2018-birth_year) FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) = 3;
'''

table = get_data(query)
table

,max,Unnamed: 1
0,78,NaN


## Aggregation: find the start station id that has the highest number of bikes taken from it

In [11]:
query = '''
SELECT start_station_id, COUNT(bikeid) as bikes FROM fb55.citibike
GROUP BY start_station_id
ORDER BY bikes DESC
LIMIT 1;
'''

table = get_data(query)
table

# or you can sort by using MAX OR table.head(1)! both works

,start_station_id,bikes
0,521,530


## Show top 3 end station id with largest total trip duration:

In [12]:
query = '''
SELECT end_station_id, SUM(tripduration) as sum FROM fb55.citibike
GROUP BY end_station_id
ORDER BY sum DESC;
'''

table = get_data(query)
table.head(3)

,end_station_id,sum
0,477,395161
1,521,355829
2,318,320525


## Find start station id with shortest average trip duration during 1am:

In [13]:
query = '''
SELECT start_station_id, AVG(tripduration) as avg
FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) = 1
GROUP BY start_station_id
ORDER BY avg ASC
LIMIT 1;
'''

table = get_data(query)
table

,start_station_id,avg
0,316,77.0


# Mr. Vo's Lab Tasks

## Sort data by start_station_id and tripduration, with duration <= 3 hours

In [14]:
query = '''
SELECT * FROM fb55.citibike
WHERE tripduration <= 10800
ORDER BY start_station_id ASC, tripduration ASC;
'''

table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9059,-73.993929,107,2015-02-03 10:10:00+00,2015-02-03 10:12:00+00,72,8998,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19039,Subscriber,1980.0,2
1,NaN,W 52 St & 11 Ave,NaN,40772,-73.993929,189,2015-02-07 10:40:00+00,2015-02-07 10:43:00+00,72,40759,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,18131,Subscriber,1959.0,2
2,NaN,W 52 St & 11 Ave,NaN,40770,-73.993929,229,2015-02-07 10:40:00+00,2015-02-07 10:44:00+00,72,40757,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19729,Subscriber,1995.0,1
3,NaN,W 52 St & 11 Ave,NaN,22002,-73.993929,248,2015-02-05 06:19:00+00,2015-02-05 06:24:00+00,72,21948,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,16879,Subscriber,1970.0,1
4,NaN,W 52 St & 11 Ave,NaN,43107,-73.993929,252,2015-02-07 15:43:00+00,2015-02-07 15:47:00+00,72,43099,40.767272,515,W 43 St & 10 Ave,40.760094,-73.994618,15277,Subscriber,1983.0,1
5,NaN,W 52 St & 11 Ave,NaN,32603,-73.993929,261,2015-02-06 08:58:00+00,2015-02-06 09:02:00+00,72,32566,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,18492,Subscriber,1971.0,1
6,NaN,W 52 St & 11 Ave,NaN,16015,-73.993929,276,2015-02-04 12:46:00+00,2015-02-04 12:51:00+00,72,15971,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,19216,Subscriber,1961.0,1
7,NaN,W 52 St & 11 Ave,NaN,41366,-73.993929,283,2015-02-07 12:10:00+00,2015-02-07 12:15:00+00,72,41355,40.767272,447,8 Ave & W 52 St,40.763707,-73.985162,21588,Subscriber,1974.0,1
8,NaN,W 52 St & 11 Ave,NaN,42084,-73.993929,288,2015-02-07 13:50:00+00,2015-02-07 13:54:00+00,72,42074,40.767272,514,12 Ave & W 40 St,40.760875,-74.002777,19124,Subscriber,1978.0,1
9,NaN,W 52 St & 11 Ave,NaN,2348,-73.993929,301,2015-02-01 13:08:00+00,2015-02-01 13:13:00+00,72,2282,40.767272,447,8 Ave & W 52 St,40.763707,-73.985162,18174,Subscriber,1974.0,1


## Top 10 Records

In [15]:
query = '''
SELECT * FROM fb55.citibike
ORDER BY cartodb_id ASC
LIMIT 10;
'''

table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,8 Ave & W 31 St,NaN,70,-73.994811,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,1,40.750450,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978,2
1,NaN,E 17 St & Broadway,NaN,71,-73.990093,379,2015-02-01 00:00:00+00,2015-02-01 00:07:00+00,497,2,40.737050,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993,1
2,NaN,Grand Army Plaza & Central Park S,NaN,72,-73.973715,2474,2015-02-01 00:01:00+00,2015-02-01 00:42:00+00,281,3,40.764397,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969,2
3,NaN,6 Ave & Broome St,NaN,73,-74.004704,818,2015-02-01 00:01:00+00,2015-02-01 00:15:00+00,2004,4,40.724399,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985,2
4,NaN,Lawrence St & Willoughby St,NaN,74,-73.986317,544,2015-02-01 00:01:00+00,2015-02-01 00:10:00+00,323,5,40.692362,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,19868,Subscriber,1957,1
5,NaN,Willoughby Ave & Walworth St,NaN,75,-73.953820,717,2015-02-01 00:02:00+00,2015-02-01 00:14:00+00,373,6,40.693317,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,15854,Subscriber,1979,1
6,NaN,W 56 St & 6 Ave,NaN,76,-73.977225,1306,2015-02-01 00:04:00+00,2015-02-01 00:26:00+00,352,7,40.763406,504,1 Ave & E 15 St,40.732219,-73.981656,15173,Subscriber,1983,1
7,NaN,E 4 St & 2 Ave,NaN,77,-73.989780,913,2015-02-01 00:04:00+00,2015-02-01 00:19:00+00,439,8,40.726281,116,W 17 St & 8 Ave,40.741776,-74.001497,17862,Subscriber,1955,1
8,NaN,Washington Pl & Broadway,NaN,78,-73.994046,759,2015-02-01 00:04:00+00,2015-02-01 00:17:00+00,335,9,40.729039,2012,E 27 St & 1 Ave,40.739445,-73.976806,21183,Subscriber,1985,2
9,NaN,Greenwich Ave & 8 Ave,NaN,79,-74.002638,585,2015-02-01 00:05:00+00,2015-02-01 00:15:00+00,284,10,40.739017,444,Broadway & W 24 St,40.742354,-73.989151,14843,Subscriber,1982,1


## Last 10 Records

In [16]:
query = '''
SELECT * FROM fb55.citibike
ORDER BY cartodb_id DESC
LIMIT 10;
'''

table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 4 St & 2 Ave,NaN,46199,-73.989780,917,2015-02-07 23:59:00+00,2015-02-08 00:15:00+00,439,46200,40.726281,417,Barclay St & Church St,40.712912,-74.010202,20998,Subscriber,1965,2
1,NaN,Carmine St & 6 Ave,NaN,46198,-74.002150,548,2015-02-07 23:58:00+00,2015-02-08 00:08:00+00,368,46199,40.730386,334,W 20 St & 7 Ave,40.742388,-73.997262,19540,Subscriber,1983,2
2,NaN,5 Ave & E 29 St,NaN,46197,-73.986831,392,2015-02-07 23:57:00+00,2015-02-08 00:03:00+00,474,46198,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,15545,Subscriber,1986,1
3,NaN,5 Ave & E 29 St,NaN,46196,-73.986831,428,2015-02-07 23:57:00+00,2015-02-08 00:04:00+00,474,46197,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,16395,Subscriber,1986,2
4,NaN,W 20 St & 8 Ave,NaN,46195,-74.000040,689,2015-02-07 23:57:00+00,2015-02-08 00:08:00+00,470,46196,40.743453,325,E 19 St & 3 Ave,40.736245,-73.984738,15585,Subscriber,1953,1
5,NaN,1 Ave & E 44 St,NaN,46194,-73.969053,1422,2015-02-07 23:57:00+00,2015-02-08 00:20:00+00,455,46195,40.750020,265,Stanton St & Chrystie St,40.722293,-73.991475,20184,Subscriber,1960,2
6,NaN,E 20 St & 2 Ave,NaN,46193,-73.982050,993,2015-02-07 23:56:00+00,2015-02-08 00:13:00+00,461,46194,40.735877,295,Pike St & E Broadway,40.714067,-73.992939,16722,Subscriber,1974,1
7,NaN,Warren St & Church St,NaN,46192,-74.009106,1165,2015-02-07 23:56:00+00,2015-02-08 00:15:00+00,152,46193,40.714740,325,E 19 St & 3 Ave,40.736245,-73.984738,16978,Subscriber,1959,1
8,NaN,W 47 St & 10 Ave,NaN,46191,-73.993012,1248,2015-02-07 23:56:00+00,2015-02-08 00:17:00+00,495,46192,40.762699,432,E 7 St & Avenue A,40.726218,-73.983799,16300,Subscriber,1984,1
9,NaN,W 17 St & 8 Ave,NaN,46190,-74.001497,306,2015-02-07 23:55:00+00,2015-02-08 00:01:00+00,116,46191,40.741776,494,W 26 St & 8 Ave,40.747348,-73.997236,14736,Subscriber,1983,2


## Unique Start Station ID Values

In [17]:
query = '''
SELECT DISTINCT start_station_id FROM fb55.citibike;
'''

table = get_data(query)
table

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN
5,144,NaN
6,266,NaN
7,263,NaN
8,216,NaN
9,328,NaN


## Count number of trips,  Average, Minimum, and Maximum Trip Duration

In [18]:
query = '''
SELECT COUNT(tripduration), AVG(tripduration), MIN(tripduration), MAX(tripduration)
FROM fb55.citibike;
'''

table = get_data(query)
table

,count,avg,min,max
0,46200,675.865823,60,43016


## Selecting trips started on Feb-02-2015 only 

In [19]:
query = '''
SELECT * FROM fb55.citibike
WHERE EXTRACT(YEAR FROM starttime) = 2015 AND
EXTRACT(MONTH FROM starttime) = 02 AND
EXTRACT(DAY FROM starttime) = 02;
'''

table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2
5,NaN,W 33 St & 7 Ave,NaN,1037,-73.990931,498,2015-02-02 06:20:00+00,2015-02-02 06:28:00+00,492,6526,40.750200,493,W 45 St & 6 Ave,40.756800,-73.982912,16013,Subscriber,1962.0,2
6,NaN,W 27 St & 7 Ave,NaN,6510,-73.993915,199,2015-02-02 00:02:00+00,2015-02-02 00:05:00+00,442,6442,40.746647,489,10 Ave & W 28 St,40.750664,-74.001768,20684,Subscriber,1992.0,1
7,NaN,E 11 St & 1 Ave,NaN,6511,-73.984267,418,2015-02-02 00:03:00+00,2015-02-02 00:10:00+00,326,6443,40.729538,349,Rivington St & Ridge St,40.718502,-73.983299,16094,Subscriber,1964.0,2
8,NaN,Peck Slip & Front St,NaN,6512,-74.001670,276,2015-02-02 00:04:00+00,2015-02-02 00:09:00+00,279,6444,40.707873,415,Pearl St & Hanover Square,40.704718,-74.009260,17362,Subscriber,1974.0,1
9,NaN,E 16 St & 5 Ave,NaN,6513,-73.992390,420,2015-02-02 00:06:00+00,2015-02-02 00:13:00+00,496,6445,40.737262,237,E 11 St & 2 Ave,40.730473,-73.986724,15475,Subscriber,1992.0,2


## Selecting trips started on the weekends 

In [25]:
query = '''
SELECT * FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime) IN (0, 6);
'''

table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0
5,NaN,E 20 St & 2 Ave,NaN,39894,-73.982050,296,2015-02-07 08:13:00+00,2015-02-07 08:18:00+00,461,39876,40.735877,435,W 21 St & 6 Ave,40.741740,-73.994156,17278,Subscriber,1987.0,1
6,NaN,Broadway & W 60 St,NaN,41512,-73.981918,600,2015-02-07 12:35:00+00,2015-02-07 12:45:00+00,499,41501,40.769155,523,W 38 St & 8 Ave,40.754666,-73.991382,15224,Subscriber,1959.0,1
7,NaN,E 6 St & Avenue D,NaN,41595,-73.976687,1203,2015-02-07 12:47:00+00,2015-02-07 13:07:00+00,411,41585,40.722281,428,E 3 St & 1 Ave,40.724677,-73.987834,21461,Subscriber,1968.0,1
8,NaN,Great Jones St,NaN,41806,-73.993790,520,2015-02-07 13:16:00+00,2015-02-07 13:25:00+00,229,41796,40.727434,168,W 18 St & 6 Ave,40.739713,-73.994564,14621,Subscriber,1964.0,1
9,NaN,W 13 St & 6 Ave,NaN,42316,-73.997044,487,2015-02-07 14:18:00+00,2015-02-07 14:26:00+00,345,42307,40.736494,509,9 Ave & W 22 St,40.745497,-74.001971,21362,Subscriber,1962.0,1


## Average trip duration during weekends

In [26]:
query = '''
SELECT AVG(tripduration) FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime) IN (0, 6);
'''

table = get_data(query)
table

,avg,Unnamed: 1
0,662.942181,NaN


## Selecting trips started on weekdays

In [27]:
query = '''
SELECT * FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime) IN (1,2,3,4,5);
'''

table = get_data(query)
table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 39 St & 3 Ave,NaN,8327,-73.976049,441,2015-02-03 08:02:00+00,2015-02-03 08:10:00+00,167,8263,40.748901,526,E 33 St & 5 Ave,40.747659,-73.984907,21518,Subscriber,1975.0,1
3,NaN,10 Ave & W 28 St,NaN,9435,-74.001768,1475,2015-02-03 12:31:00+00,2015-02-03 12:56:00+00,489,9375,40.750664,455,1 Ave & E 44 St,40.750020,-73.969053,19863,Subscriber,1984.0,2
4,NaN,E 2 St & 2 Ave,NaN,10472,-73.990697,420,2015-02-03 16:45:00+00,2015-02-03 16:52:00+00,403,10411,40.725029,297,E 15 St & 3 Ave,40.734232,-73.986923,19257,Subscriber,1984.0,1
5,NaN,1 Ave & E 44 St,NaN,10931,-73.969053,876,2015-02-03 17:37:00+00,2015-02-03 17:52:00+00,455,10867,40.750020,477,W 41 St & 8 Ave,40.756405,-73.990026,17392,Subscriber,1977.0,1
6,NaN,W 45 St & 6 Ave,NaN,11361,-73.982912,640,2015-02-03 18:21:00+00,2015-02-03 18:32:00+00,493,11300,40.756800,440,E 45 St & 3 Ave,40.752554,-73.972826,21433,Subscriber,1981.0,1
7,NaN,W 18 St & 6 Ave,NaN,12173,-73.994564,571,2015-02-03 20:26:00+00,2015-02-03 20:35:00+00,168,12114,40.739713,494,W 26 St & 8 Ave,40.747348,-73.997236,19493,Subscriber,1964.0,1
8,NaN,E 58 St & 3 Ave,NaN,12211,-73.967245,821,2015-02-03 20:37:00+00,2015-02-03 20:50:00+00,305,12152,40.760958,515,W 43 St & 10 Ave,40.760094,-73.994618,16564,Subscriber,1983.0,1
9,NaN,E 33 St & 5 Ave,NaN,12560,-73.984907,376,2015-02-03 22:20:00+00,2015-02-03 22:26:00+00,526,12500,40.747659,507,E 25 St & 2 Ave,40.739126,-73.979738,19721,Subscriber,1991.0,1


## Average trip duration on weekdays

In [28]:
query = '''
SELECT AVG(tripduration) FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime) IN (1,2,3,4,5);
'''

table = get_data(query)
table

,avg,Unnamed: 1
0,681.052292,NaN


## Showing the list of start station locations 

In [30]:
query = '''
SELECT start_station_id FROM fb55.citibike
GROUP BY start_station_id;
'''

table = get_data(query)
table

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN
5,144,NaN
6,266,NaN
7,263,NaN
8,216,NaN
9,328,NaN


## Showing the number of trips started per station

In [31]:
query = '''
SELECT start_station_id, COUNT(start_station_id)
FROM fb55.citibike
GROUP BY start_station_id;
'''

table = get_data(query)
table

,start_station_id,count
0,120,17
1,285,373
2,251,194
3,195,137
4,453,218
5,144,7
6,266,38
7,263,217
8,216,12
9,328,73


## Showing the number of trips started per station for stations within 500m of Time Square

In [32]:
# Each degree of latitude is approximately 69 miles (111 kilometers) apart, so 500m means a change of approx 0.0045
# Assume longitude also changes in the same way approximately
query = '''
SELECT start_station_id, COUNT(start_station_id) 
FROM fb55.citibike
WHERE start_station_latitude <= 40.7622 AND
start_station_latitude > 40.7532 AND
start_station_longitude <= -73.9812 AND
start_station_longitude > -73.9902
GROUP BY start_station_id;
'''

table = get_data(query)
table

,start_station_id,count
0,173,213
1,493,141
2,2021,141
3,524,112
4,477,507
5,465,251


## Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare

In [37]:
query = '''
SELECT start_station_id, AVG(tripduration) 
FROM fb55.citibike
WHERE start_station_latitude <= 40.7622 AND
start_station_latitude > 40.7532 AND
start_station_longitude <= -73.9812 AND
start_station_longitude > -73.9902 AND
EXTRACT(DOW FROM starttime) IN (0, 6)
GROUP BY start_station_id
ORDER BY avg DESC
LIMIT 1;
'''

table = get_data(query)
table

,start_station_id,avg
0,173,1010.104167


## Extra: Create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours

In [44]:
query = '''
SELECT start_station_id, end_station_id, COUNT(start_station_id)
FROM fb55.citibike
WHERE start_station_latitude <= 40.7622 AND
start_station_latitude > 40.7532 AND
start_station_longitude <= -73.9812 AND
start_station_longitude > -73.9902 AND
tripduration < 7200
GROUP BY start_station_id, end_station_id
ORDER BY count DESC;
'''

table = get_data(query)
table

,start_station_id,end_station_id,count
0,477,318,31
1,477,359,21
2,477,517,17
3,477,519,15
4,173,318,15
5,524,318,14
6,477,153,14
7,493,492,14
8,465,492,14
9,477,478,13
